In [1]:
# Importujemy biblioteki
import pandas as pd
import numpy as np
import os
import ast
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

Ewaluacja danych 70_30 dla PNN

In [9]:
# Ścieżki
test_texts_path = '../data/70_30/test_texts_w2v.csv'
test_labels_path = '../data/70_30/test_labels.csv'
model_path = '../models/w2v_LSTM_70_30_PNN.h5'
label_encoder_path = '../models/w2v_LSTM_70_30_PNN_label_encoder.pkl'
evaluation_dir = '../evaluation/eval_w2v_LSTM_70_30_PNN'
os.makedirs(evaluation_dir, exist_ok=True)

# Wczytujemy model
model = load_model(model_path)

print("Model został wczytany.")

# Wczytujemy label encoder
with open(label_encoder_path, 'rb') as le_file:
    label_encoder = pickle.load(le_file)

print("LabelEncoder został wczytany.")

# Wczytujemy dane testowe
X_test_raw = pd.read_csv(test_texts_path, index_col=0)
y_test = pd.read_csv(test_labels_path, index_col=0).squeeze()

print(f"Wczytano dane testowe: {X_test_raw.shape[0]} rekordów.")

# Rozkład klas w zbiorze testowym (przed mapowaniem)
print("Rozkład klas w zbiorze testowym (przed mapowaniem):")
print(y_test.value_counts(normalize=True))

# Mapujemy gwiazdki na klasy sentymentu
def map_sentiment(star_rating):
    if star_rating in [4, 5]:
        return 'Pozytywna'
    elif star_rating == 3:
        return 'Neutralna'                         
    else:
        return 'Negatywna'

y_test = y_test.map(map_sentiment)

print("Rozkład klas w zbiorze testowym (po mapowaniu):")
print(y_test.value_counts(normalize=True))

# Zapisujemy rozkład klas
class_distribution = y_test.value_counts(normalize=True)
class_distribution.to_csv(os.path.join(evaluation_dir, 'class_distribution_test_70_30.csv'))

# Wyrównujemy indeksy
X_test_raw, y_test = X_test_raw.align(y_test, join='inner', axis=0)

# Parsujemy dane testowe
X_test_list = X_test_raw['text'].apply(ast.literal_eval).apply(np.array).tolist()

# Padding sekwencji
X_test_padded = pad_sequences(X_test_list, padding='post', dtype='float32')

print(f"Kształt danych testowych po paddingu: {X_test_padded.shape}")

# Predykcja
y_pred_proba = model.predict(X_test_padded)
y_pred = y_pred_proba.argmax(axis=1)
y_pred_labels = label_encoder.inverse_transform(y_pred)

print("Predykcja zakończona.")

# Obliczamy metryki
accuracy = accuracy_score(y_test, y_pred_labels)
precision = precision_score(y_test, y_pred_labels, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred_labels, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred_labels, average='weighted', zero_division=0)
conf_matrix = confusion_matrix(y_test, y_pred_labels, labels=label_encoder.classes_)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Tworzymy podsumowanie metryk
metrics_summary = pd.DataFrame({
    'metric': ['accuracy', 'precision', 'recall', 'f1_score'],
    'value': [accuracy, precision, recall, f1]
})

# Zapisujemy podsumowanie do pliku CSV
metrics_summary.to_csv(os.path.join(evaluation_dir, 'metrics_summary_70_30.csv'), index=False)

# Confusion matrix zapisujemy osobno
conf_matrix_df = pd.DataFrame(
    conf_matrix,
    index=label_encoder.classes_,
    columns=label_encoder.classes_
)
conf_matrix_df.to_csv(os.path.join(evaluation_dir, 'confusion_matrix_70_30.csv'), index=True)

print(f"Wyniki ewaluacji zapisane w folderze: {evaluation_dir}")


Model został wczytany.
LabelEncoder został wczytany.
Wczytano dane testowe: 29999 rekordów.
Rozkład klas w zbiorze testowym (przed mapowaniem):
stars
5    0.444867
4    0.254133
3    0.113767
1    0.109133
2    0.078100
Name: proportion, dtype: float64
Rozkład klas w zbiorze testowym (po mapowaniu):
stars
Pozytywna    0.699000
Negatywna    0.187233
Neutralna    0.113767
Name: proportion, dtype: float64
Kształt danych testowych po paddingu: (29999, 500, 50)
938/938 ━━━━━━━━━━━━━━━━━━━━ 31s 32ms/step
Predykcja zakończona.
Accuracy: 0.7076902563418781
Precision: 0.6482241652049239
Recall: 0.7076902563418781
F1 Score: 0.6144421466195277
Wyniki ewaluacji zapisane w folderze: ../evaluation/eval_w2v_LSTM_70_30_PNN


Ewaluacja danych 80_20 dla PNN

In [5]:
# Ścieżki
test_texts_path = '../data/80_20/test_texts_w2v.csv'
test_labels_path = '../data/80_20/test_labels.csv'
model_path = '../models/w2v_LSTM_80_20_PNN.h5'
label_encoder_path = '../models/w2v_LSTM_80_20_PNN_label_encoder.pkl'
evaluation_dir = '../evaluation/eval_w2v_LSTM_80_20_PNN'
os.makedirs(evaluation_dir, exist_ok=True)

# Wczytujemy model
model = load_model(model_path)

print("Model został wczytany.")

# Wczytujemy label encoder
with open(label_encoder_path, 'rb') as le_file:
    label_encoder = pickle.load(le_file)

print("LabelEncoder został wczytany.")

# Wczytujemy dane testowe
X_test_raw = pd.read_csv(test_texts_path, index_col=0)
y_test = pd.read_csv(test_labels_path, index_col=0).squeeze()

print(f"Wczytano dane testowe: {X_test_raw.shape[0]} rekordów.")

# Rozkład klas w zbiorze testowym (przed mapowaniem)
print("Rozkład klas w zbiorze testowym (przed mapowaniem):")
print(y_test.value_counts(normalize=True))

# Mapujemy gwiazdki na klasy sentymentu
def map_sentiment(star_rating):
    if star_rating in [4, 5]:
        return 'Pozytywna'
    elif star_rating == 3:
        return 'Neutralna'
    else:
        return 'Negatywna'

y_test = y_test.map(map_sentiment)

print("Rozkład klas w zbiorze testowym (po mapowaniu):")
print(y_test.value_counts(normalize=True))

# Zapisujemy rozkład klas
class_distribution = y_test.value_counts(normalize=True)
class_distribution.to_csv(os.path.join(evaluation_dir, 'class_distribution_test_80_20.csv'))

# Wyrównujemy indeksy
X_test_raw, y_test = X_test_raw.align(y_test, join='inner', axis=0)

# Parsujemy dane testowe
X_test_list = X_test_raw['text'].apply(ast.literal_eval).apply(np.array).tolist()

# Padding sekwencji
X_test_padded = pad_sequences(X_test_list, padding='post', dtype='float32')

print(f"Kształt danych testowych po paddingu: {X_test_padded.shape}")

# Predykcja
y_pred_proba = model.predict(X_test_padded)
y_pred = y_pred_proba.argmax(axis=1)
y_pred_labels = label_encoder.inverse_transform(y_pred)

print("Predykcja zakończona.")

# Obliczamy metryki
accuracy = accuracy_score(y_test, y_pred_labels)
precision = precision_score(y_test, y_pred_labels, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred_labels, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred_labels, average='weighted', zero_division=0)
conf_matrix = confusion_matrix(y_test, y_pred_labels, labels=label_encoder.classes_)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Tworzymy podsumowanie metryk
metrics_summary = pd.DataFrame({
    'metric': ['accuracy', 'precision', 'recall', 'f1_score'],
    'value': [accuracy, precision, recall, f1]
})

# Zapisujemy podsumowanie do pliku CSV
metrics_summary.to_csv(os.path.join(evaluation_dir, 'metrics_summary_80_20.csv'), index=False)

# Confusion matrix zapisujemy osobno
conf_matrix_df = pd.DataFrame(
    conf_matrix,
    index=label_encoder.classes_,
    columns=label_encoder.classes_
)
conf_matrix_df.to_csv(os.path.join(evaluation_dir, 'confusion_matrix_80_20.csv'), index=True)

print(f"Wyniki ewaluacji zapisane w folderze: {evaluation_dir}")

Model został wczytany.
LabelEncoder został wczytany.
Wczytano dane testowe: 19999 rekordów.
Rozkład klas w zbiorze testowym (przed mapowaniem):
stars
5    0.44355
4    0.25460
3    0.11375
1    0.10950
2    0.07860
Name: proportion, dtype: float64
Rozkład klas w zbiorze testowym (po mapowaniu):
stars
Pozytywna    0.69815
Negatywna    0.18810
Neutralna    0.11375
Name: proportion, dtype: float64
Kształt danych testowych po paddingu: (19999, 475, 50)
625/625 ━━━━━━━━━━━━━━━━━━━━ 18s 28ms/step
Predykcja zakończona.
Accuracy: 0.6734836741837091
Precision: 0.5143140518510328
Recall: 0.6734836741837091
F1 Score: 0.5658320449173618
Wyniki ewaluacji zapisane w folderze: ../evaluation/eval_w2v_LSTM_80_20_PNN


ewaluacja danych 70-30 5 gwiazdek

In [ ]:
# Ścieżki
test_texts_path = '../data/70_30/test_texts_w2v.csv'
test_labels_path = '../data/70_30/test_labels.csv'
model_path = '../models/w2v_LSTM_70_30_stars.h5'
label_encoder_path = '../models/w2v_LSTM_70_30_stars_label_encoder.pkl'
evaluation_dir = '../evaluation/eval_w2v_LSTM_70_30_stars'
os.makedirs(evaluation_dir, exist_ok=True)

# Wczytujemy model
model = load_model(model_path)
print("Model został wczytany.")

# Wczytujemy label encoder
with open(label_encoder_path, 'rb') as le_file:
    label_encoder = pickle.load(le_file)
print("LabelEncoder został wczytany.")

# Wczytujemy dane testowe
X_test_raw = pd.read_csv(test_texts_path, index_col=0)
y_test = pd.read_csv(test_labels_path, index_col=0).squeeze()
print(f"Wczytano dane testowe: {X_test_raw.shape[0]} rekordów.")

# Rozkład klas w zbiorze testowym
print("Rozkład klas w zbiorze testowym (gwiazdki):")
print(y_test.value_counts(normalize=True))

# Zapisujemy rozkład klas
class_distribution = y_test.value_counts(normalize=True)
class_distribution.to_csv(os.path.join(evaluation_dir, 'class_distribution_test_70_30.csv'))
print("Zapisano rozkład klas do pliku.")

# Wyrównujemy indeksy
X_test_raw, y_test = X_test_raw.align(y_test, join='inner', axis=0)
print(f"Po align: X_test_raw shape: {X_test_raw.shape}, y_test shape: {y_test.shape}")

# Parsowanie danych testowych
X_test_list = X_test_raw['text'].apply(ast.literal_eval).apply(np.array).tolist()

# Padding sekwencji
X_test_padded = pad_sequences(X_test_list, padding='post', dtype='float32')
print(f"Kształt danych testowych po paddingu: {X_test_padded.shape}")

# Predykcja
y_pred_proba = model.predict(X_test_padded)
y_pred = y_pred_proba.argmax(axis=1)
y_pred_labels = label_encoder.inverse_transform(y_pred)

print("Predykcja zakończona.")

# Rozkład klas w predykcji
print("Rozkład klas w predykcji (gwiazdki):")
print(pd.Series(y_pred_labels).value_counts(normalize=True))

# Obliczamy metryki
accuracy = accuracy_score(y_test, y_pred_labels)
precision = precision_score(y_test, y_pred_labels, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred_labels, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred_labels, average='weighted', zero_division=0)
conf_matrix = confusion_matrix(y_test, y_pred_labels, labels=label_encoder.classes_)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Tworzymy podsumowanie metryk
metrics_summary = pd.DataFrame({
    'metric': ['accuracy', 'precision', 'recall', 'f1_score'],
    'value': [accuracy, precision, recall, f1]
})

# Zapisujemy podsumowanie do pliku CSV
metrics_summary.to_csv(os.path.join(evaluation_dir, 'metrics_summary_70_30.csv'), index=False)
print("Zapisano metrics_summary_70_30.csv.")

# Confusion matrix zapisujemy osobno
conf_matrix_df = pd.DataFrame(
    conf_matrix,
    index=label_encoder.classes_,
    columns=label_encoder.classes_
)
conf_matrix_df.to_csv(os.path.join(evaluation_dir, 'confusion_matrix_70_30.csv'), index=True)
print("Zapisano confusion_matrix_70_30.csv.")

print(f"Wyniki ewaluacji zapisane w folderze: {evaluation_dir}")

Model został wczytany.
LabelEncoder został wczytany.
Wczytano dane testowe: 29999 rekordów.
Rozkład klas w zbiorze testowym (gwiazdki):
stars
5    0.444867
4    0.254133
3    0.113767
1    0.109133
2    0.078100
Name: proportion, dtype: float64
Zapisano rozkład klas do pliku.
Po align: X_test_raw shape: (29999, 1), y_test shape: (29999,)


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x0000024569E6DD80>>
Traceback (most recent call last):
  File "c:\Users\Maciej\Desktop\sem6\UCZENIE MASZYNOWE\sentiment_analysis\.venv\lib\site-packages\ipykernel\ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


ewaluacja danych 80-20 5 gwiazdek

In [7]:
# Ścieżki
test_texts_path = '../data/80_20/test_texts_w2v.csv'
test_labels_path = '../data/80_20/test_labels.csv'
model_path = '../models/w2v_LSTM_80_20_stars.h5'
label_encoder_path = '../models/w2v_LSTM_80_20_stars_label_encoder.pkl'
evaluation_dir = '../evaluation/eval_w2v_LSTM_80_20_stars'
os.makedirs(evaluation_dir, exist_ok=True)

# Wczytujemy model
model = load_model(model_path)
print("Model został wczytany.")

# Wczytujemy label encoder
with open(label_encoder_path, 'rb') as le_file:
    label_encoder = pickle.load(le_file)
print("LabelEncoder został wczytany.")

# Wczytujemy dane testowe
X_test_raw = pd.read_csv(test_texts_path, index_col=0)
y_test = pd.read_csv(test_labels_path, index_col=0).squeeze()
print(f"Wczytano dane testowe: {X_test_raw.shape[0]} rekordów.")

# Rozkład klas w zbiorze testowym
print("Rozkład klas w zbiorze testowym (gwiazdki):")
print(y_test.value_counts(normalize=True))

# Zapisujemy rozkład klas
class_distribution = y_test.value_counts(normalize=True)
class_distribution.to_csv(os.path.join(evaluation_dir, 'class_distribution_test_80_20.csv'))
print("Zapisano rozkład klas do pliku.")

# Wyrównujemy indeksy
X_test_raw, y_test = X_test_raw.align(y_test, join='inner', axis=0)
print(f"Po align: X_test_raw shape: {X_test_raw.shape}, y_test shape: {y_test.shape}")

# Parsowanie danych testowych
X_test_list = X_test_raw['text'].apply(ast.literal_eval).apply(np.array).tolist()

# Padding sekwencji
X_test_padded = pad_sequences(X_test_list, padding='post', dtype='float32')
print(f"Kształt danych testowych po paddingu: {X_test_padded.shape}")

# Predykcja
y_pred_proba = model.predict(X_test_padded)
y_pred = y_pred_proba.argmax(axis=1)
y_pred_labels = label_encoder.inverse_transform(y_pred)

print("Predykcja zakończona.")

# Rozkład klas w predykcji
print("Rozkład klas w predykcji (gwiazdki):")
print(pd.Series(y_pred_labels).value_counts(normalize=True))

# Obliczamy metryki
accuracy = accuracy_score(y_test, y_pred_labels)
precision = precision_score(y_test, y_pred_labels, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred_labels, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred_labels, average='weighted', zero_division=0)
conf_matrix = confusion_matrix(y_test, y_pred_labels, labels=label_encoder.classes_)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Tworzymy podsumowanie metryk
metrics_summary = pd.DataFrame({
    'metric': ['accuracy', 'precision', 'recall', 'f1_score'],
    'value': [accuracy, precision, recall, f1]
})

# Zapisujemy podsumowanie do pliku CSV
metrics_summary.to_csv(os.path.join(evaluation_dir, 'metrics_summary_80_20.csv'), index=False)
print("Zapisano metrics_summary_80_20.csv.")

# Confusion matrix zapisujemy osobno
conf_matrix_df = pd.DataFrame(
    conf_matrix,
    index=label_encoder.classes_,
    columns=label_encoder.classes_
)
conf_matrix_df.to_csv(os.path.join(evaluation_dir, 'confusion_matrix_80_20.csv'), index=True)
print("Zapisano confusion_matrix_80_20.csv.")

print(f"Wyniki ewaluacji zapisane w folderze: {evaluation_dir}")

Model został wczytany.
LabelEncoder został wczytany.
Wczytano dane testowe: 19999 rekordów.
Rozkład klas w zbiorze testowym (gwiazdki):
stars
5    0.44355
4    0.25460
3    0.11375
1    0.10950
2    0.07860
Name: proportion, dtype: float64
Zapisano rozkład klas do pliku.
Po align: X_test_raw shape: (19999, 1), y_test shape: (19999,)
Kształt danych testowych po paddingu: (19999, 475, 50)
625/625 ━━━━━━━━━━━━━━━━━━━━ 19s 29ms/step
Predykcja zakończona.
Rozkład klas w predykcji (gwiazdki):
5    0.809990
4    0.142707
1    0.026551
3    0.016851
2    0.003900
Name: proportion, dtype: float64
Accuracy: 0.3962
Precision: 0.2856
Recall: 0.3962
F1 Score: 0.3069
Zapisano metrics_summary_80_20.csv.
Zapisano confusion_matrix_80_20.csv.
Wyniki ewaluacji zapisane w folderze: ../evaluation/eval_w2v_LSTM_80_20_stars
